# 🚀 WandB 튜토리얼
WandB 기초를 위한 튜토리얼 노트북입니다.  
MNIST data를 활용하여 간단한 실습을 진행할 예정이며,  
wandb watch와 log등 대시보드를 활용하기 위한 준비단계입니다.  
자세한 내용은 [1. WandB란? - 강력한 MLOps Tool](https://pebpung.github.io/wandb/2021/10/06/WandB-1.html)를 참고하시면 됩니다. 

### 0️⃣ Step 0: W&B 설치 후 로그인
WandB를 설치한 후 로그인을 합니다.  
User setting -> API key -> copy & paste

In [ ]:
%%capture
!pip install wandb -qU

In [ ]:
# Log in to your W&B account
import wandb
wandb.login()

### 2️⃣ Step 1: 모델 학습을 위한 준비
이제 본격적으로 모델을 학습 시키기 전에 몇가지 준비를 하려고 합니다. 
1. 필요한 module import  
2. hyper-parameter config 설정
3. dataloader 정의
4. model 정의

### 1) 필요한 module import 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
from tqdm import tqdm

### 2) hyper-parameter config 설정

In [ ]:
config  = {
    'epochs': 5,
    'classes':10,
    'batch_size': 128,
    'kernels': [16, 32],
    'weight_decay': 0.0005,
    'learning_rate': 1e-3,
    'dataset': 'MNIST',
    'architecture': 'CNN',
    'val_evrey' : 5,
    'seed': 42
    }

### 3) dataloader 정의

In [ ]:
def make_loader(batch_size, train=True):
    full_dataset = datasets.MNIST(root='./data/MNIST', train=train, 
                                    download=True,  transform=transforms.ToTensor())
    
    loader = DataLoader(dataset=full_dataset,
                        batch_size=batch_size, 
                        shuffle=True,
                        pin_memory=True, num_workers=2)
    return loader

### 4) 모델 정의

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, kernels, classes=10):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, kernels[0], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, kernels[1], kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7 * 7 * kernels[-1], classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

### 3️⃣ Step 3. `train`과  `valid` 함수 정의하기


In [ ]:
def train(model, loader, criterion, optimizer, device, config):
    cumu_loss = 0
    tqdm_loader = tqdm(loader)
    for images, labels in tqdm_loader:
        images, labels = images.to(device), labels.to(device)
        logit = model(images)
        loss = criterion(logit, labels)
        cumu_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss

In [ ]:
def valid(model,test_loader):
    model.eval()
    acc_list = []
    with torch.no_grad():
        tqdm_loader = tqdm(test_loader)
        for images, labels in tqdm_loader:
            logit = model(images)
            pred = torch.argmax(logit, dim=1)
            batch_acc = accuracy_score(labels, pred)
            acc_list.append(batch_acc)
    return sum(acc_list) / len(acc_list) 

### 4️⃣  Step 4:   🏃‍♀️ WandB 실행하기
- **wandb.watch**는 gradient, topology와 관련된 정보를 visualization 하기 위한 코드입니다.
- **wandb.log**는 visualization 하고 싶은 정보를 넘겨줄 수 있습니다.

이 두가지 코드를 활용해서 gradient와 parameter를 시각화할 수 있습니다.   
wandb.watch는 모델을 선언한 후에 뒷부분에 위치 시켜줍니다.  
wandb.log는 train, validation이 끝난 후 저장합니다. 

In [ ]:
def run(config=None):
    wandb.init(project='test-pytorch', entity='pebpung', config=config)
    config = wandb.config
    device = "cuda" if torch.cuda.is_available() else "cpu"

    train_loader = make_loader(batch_size=config.batch_size, train=True)
    test_loader = make_loader(batch_size=config.batch_size, train=False)

    model = ConvNet(config.kernels, config.classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    wandb.watch(model, criterion, log="all", log_freq=10)

    for epoch in range(1, config.epochs):
        loss = train(model, train_loader, criterion, optimizer, device, config)
        print(f"[Train] Epoch {epoch:02} | Loss: {loss:.2f}")
        wandb.log({"Loss": loss})
        if epoch % config.val_evrey  == 0:
            acc = valid(model, test_loader)
            print(f"[Vaild] Epoch {epoch:02} | Acc: {acc:.3f}")
            wandb.log({"Acc": acc})
    return model

In [ ]:
model = run(config)